In [1]:
import numpy as np
import pandas as pd
import re

In [7]:
def is_no_prescale(line):
    open_idx = line.find('(')
    close_idx = line.find(')')
    digit = re.findall(r'\d+', line[open_idx:close_idx+1])
    digit_set = set(sorted([int(s) for s in digit]))
    if digit_set == {1} or digit_set == {0, 1}: 
        return True
    return False

def extract_trigger_name(line):
    open_idx = line.find(' "')
    close_idx = line.find('" ')
    trigger = line[open_idx:close_idx+1].strip().strip("\"")
    return trigger

def extract_photon_sequence(line):
    open_idx = line.find('process.HLTBeginSequence')
    close_idx = line.find('process.HLTEndSequence')
    path_list = line[open_idx:close_idx].split('+')
    ans = []
    for element in path_list:
        if "hoton" in element and "Sequence" in element:
            ans.append(element.strip())
    return ans

def find_no_prescale_photon_triggers(inlines):
    prescale_line = 'prescales = cms.vuint32'
    no_prescale_photon_triggers = []
    for i, line in enumerate(inlines):
        if prescale_line in line:
            if is_no_prescale(line):
                trigger_line = inlines[i-1]
                if 'HLT_Photon' not in trigger_line and 'HLT_Diphoton' not in trigger_line: continue
                no_prescale_photon_triggers.append(extract_trigger_name(trigger_line))
    return no_prescale_photon_triggers

def get_filter_list(inlines, sequence):
    filters = []
    for i, line in enumerate(inlines):
        if sequence in line and 'cms.Sequence(' in line:
            begin_idx = line.find('(')
            close_idx = line.find(' )')
            filter_list = line[begin_idx+2:close_idx].split(' + ')
            return filter_list
        
def get_last_filters(filter_list):
    if not any("LastFilter" in ft for ft in filter_list):
        return [filter_list[-1]]
    ans = []
    for ft in filter_list:
        if "LastFilter" in ft and 'cms.ignore' not in ft:
            ans.append(ft)
    return ans

def get_filter_for_trigger(inlines, trigger_list):
    trigger_filter_dict = {}
    for i, line in enumerate(inlines):
        if 'process' in line and 'cms.Path(' in line:
            for trigger in trigger_list:
                if trigger in line:
                    trigger_filter_dict[trigger] = {}
                    sequence = extract_photon_sequence(line)
                    if len(sequence) == 0:
                        print(trigger)
                    if len(sequence) > 1:
                        print("WARNING: More than 1 photon sequences found {}".format(sequence))
                    filter_list = get_filter_list(inlines, sequence[0])
                    last_filters = get_last_filters(filter_list)
                    trigger_filter_dict[trigger][sequence[0]] = last_filters
    return trigger_filter_dict

with open('/storage/user/qnguyen/DelayedPhoton/CMSSW_10_6_6/src/DelayedPhotonID/HLTconfig.py','r') as infile:
    inlines = infile.readlines()
    no_prescale_photon_triggers = find_no_prescale_photon_triggers(inlines)
    trigger_filter_dict = get_filter_for_trigger(inlines, no_prescale_photon_triggers)
    
    for trigger in trigger_filter_dict:
        print(trigger)
        for sequence in trigger_filter_dict[trigger]:
            #print("\t{}".format(sequence))
            for ft in trigger_filter_dict[trigger][sequence]:
                print("\t{}".format(ft))
                
print("Total: {} triggers".format(len(trigger_filter_dict)))

HLT_Photon135_PFMET100_v2
	process.hltEG135HEFilter
HLT_Photon250_NoHE_v3
	process.hltEG250erEtFilter
HLT_Photon300_NoHE_v3
	process.hltEG300erEtFilter
HLT_Photon36_R9Id85_OR_CaloId24b40e_Iso50T80L_Photon22_AND_HE10_R9Id65_Eta2_Mass15_v3
	process.hltEG36RId85ORCaloId24b40eIso50T80LANDHE10R9Id65Eta2LegCombLastFilter
	process.hltDiEG22R9Id85ORCaloId24b40eIso50T80LANDHE10R9Id65Mass15CombMassLastFilter
HLT_Photon120_R9Id90_HE10_Iso40_EBOnly_PFMET40_v4
	process.hltEG120R9Id90HE10Iso40EBOnlyTrackIsoFilter
HLT_Photon120_R9Id90_HE10_Iso40_EBOnly_VBF_v3
	process.hltEG120R9Id90HE10Iso40EBOnlyTrackIsoFilter
HLT_Photon175_v4
	process.hltEG175HEFilter
HLT_Photon165_HE10_v4
	process.hltEG165HE10Filter
HLT_Photon165_R9Id90_HE10_IsoM_v4
	process.hltEG165R9Id90HE10IsoMTrackIsoFilter
HLT_Diphoton30_18_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass90_v2
	process.hltEG30LRId85ORIso60CaloId15b35eANDHE12R9Id50b80eLegCombLastFilter
	process.hltDiEG18R9Id85b90eORIso60CaloId15b35eANDHE12R9Id50b80eMass90CombMassLastFilter


In [15]:
def get_trigger_indices(trigger_list):
    reduced_list = []
    for trigger in trigger_list:
            if '_v' in trigger:
                reduced_list.append(trigger[:trigger.find('_v')])
            else:
                reduced_list.append(trigger)
                
    trigger_table = '/storage/user/qnguyen/DelayedPhoton/CMSSW_9_4_9/src/Tuplizer/DelayedPhotonTuplizer/data/RazorHLTPathnames2016.dat'
    trigger_idx = []
    with open(trigger_table, 'r') as table:
        table_lines = table.readlines()
        for i, line in enumerate(table_lines):
            for trigger in reduced_list:
                if trigger in line:
                    reduced_list.remove(trigger)
                    print(line)
                    idx = re.findall(r'\d+', line)[0]
                    trigger_idx.append(int(idx))
        print("Can't find these: ")
        for trigger in reduced_list:
            print(trigger)
        
        print("List of trigger indices to use: {}".format(trigger_idx))
        
get_trigger_indices(trigger_filter_dict)

80    HLT_Photon36_R9Id85_OR_CaloId24b40e_Iso50T80L_Photon22_AND_HE10_R9Id65_Eta2_Mass15

81    HLT_Photon42_R9Id85_OR_CaloId24b40e_Iso50T80L_Photon25_AND_HE10_R9Id65_Eta2_Mass15

82    HLT_Diphoton30_18_R9Id_OR_IsoCaloId_AND_HE_R9Id_Mass90

84    HLT_Diphoton30_18_R9Id_OR_IsoCaloId_AND_HE_R9Id_DoublePixelSeedMatch_Mass70

85    HLT_Diphoton30PV_18PV_R9Id_AND_IsoCaloId_AND_HE_R9Id_DoublePixelVeto_Mass55

86    HLT_Diphoton30_18_Solid_R9Id_AND_IsoCaloId_AND_HE_R9Id_Mass55

87    HLT_Diphoton30EB_18EB_R9Id_OR_IsoCaloId_AND_HE_R9Id_DoublePixelVeto_Mass55

101    HLT_Photon175

102    HLT_Photon165_HE10

103    HLT_Photon250_NoHE

104    HLT_Photon300_NoHE

112    HLT_Photon165_R9Id90_HE10_IsoM

116    HLT_Photon135_PFMET100

220    HLT_Photon500

221    HLT_Photon600

Can't find these: 
HLT_Photon120_R9Id90_HE10_Iso40_EBOnly_PFMET40
HLT_Photon120_R9Id90_HE10_Iso40_EBOnly_VBF
HLT_Photon90_CaloIdL_PFHT500
HLT_Photon90_CaloIdL_PFHT600
List of trigger indices to use: [80, 81, 82, 84, 85, 86, 